# 🚀 Satellite Telemetry Anomaly Detection & Predictive Maintenance
This notebook simulates satellite telemetry data, detects anomalies, predicts potential subsystem failures, and provides recommendations.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)


: 

## 1. Simulate Telemetry Data

In [ ]:

# Time index
time = pd.date_range(start="2025-01-01", periods=500, freq="T")

# Normal telemetry values
battery = np.clip(80 - 0.01*np.arange(500) + np.random.normal(0, 0.5, 500), 20, 100)
temperature = np.clip(25 + 2*np.sin(np.linspace(0, 20, 500)) + np.random.normal(0, 0.5, 500), -20, 80)
power = np.clip(100 + 10*np.sin(np.linspace(0, 10, 500)) + np.random.normal(0, 3, 500), 50, 150)
current = np.clip(5 + 0.5*np.sin(np.linspace(0, 15, 500)) + np.random.normal(0, 0.2, 500), 2, 10)

# Inject anomalies
battery[100] -= 30
temperature[250] += 40
power[400] -= 50
current[450] += 5

# Create DataFrame
df = pd.DataFrame({
    "time": time,
    "battery": battery,
    "temperature": temperature,
    "power": power,
    "current": current
}).set_index("time")

df.head()


## 2. Visualize Telemetry Data

In [ ]:

fig, axes = plt.subplots(4, 1, figsize=(12,10), sharex=True)
df["battery"].plot(ax=axes[0], title="Battery (%)")
df["temperature"].plot(ax=axes[1], title="Temperature (°C)", color="orange")
df["power"].plot(ax=axes[2], title="Power (W)", color="green")
df["current"].plot(ax=axes[3], title="Current (A)", color="red")
plt.tight_layout()
plt.show()


## 3. Correlation Analysis

In [ ]:

plt.figure(figsize=(6,5))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Telemetry Data")
plt.show()


## 4. Anomaly Detection - Threshold Based

In [ ]:

def detect_threshold_anomalies(data, col, lower, upper):
    anomalies = (data[col] < lower) | (data[col] > upper)
    return anomalies

thresholds = {
    "battery": (30, 100),
    "temperature": (-10, 60),
    "power": (60, 140),
    "current": (3, 8)
}

for col, (low, high) in thresholds.items():
    df[col+"_anomaly"] = detect_threshold_anomalies(df, col, low, high)

df[["battery_anomaly", "temperature_anomaly", "power_anomaly", "current_anomaly"]].sum()


## 5. Anomaly Detection - Machine Learning (Isolation Forest)

In [ ]:

iso = IsolationForest(contamination=0.02, random_state=42)
df["ml_anomaly"] = iso.fit_predict(df[["battery","temperature","power","current"]])
df["ml_anomaly"] = df["ml_anomaly"].map({1:0, -1:1})

print("Detected anomalies:", df["ml_anomaly"].sum())

plt.figure(figsize=(12,5))
plt.plot(df.index, df["battery"], label="Battery")
plt.scatter(df.index[df["ml_anomaly"]==1], df["battery"][df["ml_anomaly"]==1], color="red", label="Anomaly")
plt.legend()
plt.title("Battery with ML Anomalies")
plt.show()


## 6. Predictive Maintenance - Forecasting Battery Life

In [ ]:

# Forecast battery using ARIMA
model = ARIMA(df["battery"], order=(2,1,2))
fit = model.fit()
forecast = fit.forecast(steps=50)

plt.figure(figsize=(12,5))
plt.plot(df.index, df["battery"], label="Battery")
plt.plot(pd.date_range(df.index[-1], periods=50, freq="T"), forecast, label="Forecast", color="red")
plt.axhline(30, color="orange", linestyle="--", label="Critical Threshold")
plt.legend()
plt.title("Battery Forecast (ARIMA)")
plt.show()


## 7. Recommendation Engine

In [ ]:

def recommend_actions(row):
    recs = []
    if row["battery_anomaly"]:
        recs.append("Consider load shedding or entering safe mode due to low battery.")
    if row["temperature_anomaly"]:
        recs.append("High temperature detected, disable non-critical loads.")
    if row["power_anomaly"]:
        recs.append("Power anomaly, check solar panels or distribution system.")
    if row["current_anomaly"]:
        recs.append("Current spike detected, isolate faulty subsystem.")
    return recs

df["recommendations"] = df.apply(recommend_actions, axis=1)
df[["battery","temperature","power","current","recommendations"]].tail(10)


## ✅ Summary
- Simulated telemetry for battery, temperature, power, current.
- Detected anomalies using thresholds & Isolation Forest.
- Predicted battery depletion with ARIMA.
- Generated prioritized recommendations.
